<a href="https://colab.research.google.com/github/yexf308/AdvancedMachineLearning/blob/main/Hidden_Markov_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pylab inline 
from IPython.display import Image
import numpy.linalg as LA

$\def\m#1{\mathbf{#1}}$
$\def\mb#1{\mathbb{#1}}$
$\def\c#1{\mathcal{#1}}$

# Hidden Markov Models

Hidden Markov Model is an Unsupervised Machine Learning Algorithm which is part of the Graphical Models. However Hidden Markov Model (HMM) often trained using supervised learning method in case training data is available. 

## Model
<img src="https://github.com/yexf308/MAT592/blob/main/image/HMM.png?raw=true" width="500" />

Hidden Markov Models assumes that observed dataset $\{\mathbf{x}^{(t)}\}_{t=1}^N\subset \mb{R}^d$ are sampled through a generative process:

- Generate the sequence of states $\{z^{(t)}\}_{t=1}^N$ by a Markov chain whose transition matrix is $M\in \mb{R}^{K\times K}$ starting the initial distribution as the invariant distribution $\vec\pi$, where $K$ is number of hidden states. 

- **Assumption 1:** We assume the Markov chain has reached the stationary,  
 $$P(z^{(1)})=\vec\pi, \qquad \vec\pi = \vec\pi M$$

- **Assumption 2:** the state at time $t$, $z^{(t)}$, depends only on the state at the previous time $t-1$, $z^{(t-1)}$, i.e.,
 $$ P(z^{(t)}|z^{(1:t-1)})=P(z^{(t)}|z^{(t-1)})$$


- Given the state $z^{(t)}=c$, generate a point $\mathbf{x}^{(t)}$ from the associated multivariate Gaussian distribution $\mathcal{N}(\mu_c, \Sigma_c)$ with PDF. It is also called **emission distribution**:
$$p(\m{x}|\mu_c, \Sigma_c)= \frac{1}{\sqrt{(2\pi)^d|\Sigma_c|}}\exp\left(-\frac{1}{2}(\mathbf{x}-\mu_c)^\top\Sigma_c^{-1}(\mathbf{x}-\mu_c)\right)$$
 The emission distribution can be some discrete value as well. Here we just assume this is a Gaussian. 
 

 - **Assumption 3**: The observed data at time $t$, $\m{x}^{(t)}$ depends only on the state at time $t$, $z^{(t)}$, i.e.,
 $$P(\m{x}^{(t)}|z^{(1:t)}) =P(\m{x}^{(t)}|z^{(t)})$$ 


Now the parameter $\theta=\{\pi_c, \mu_c, \Sigma_c,  M_{ij}\}_{c,i,j=1}^K$

## Center problems in HMM
- **Evaluation**: Efficiently compute a wide range of
conditional probabilities given $\m{x}^{(1:N)}$: $P(z^{(t)}=k|\m{x}^{(1:N)},\theta), P(z^{(t)}=k, z^{(t+1)}=c|\m{x}^{(1:N)}, \theta)$. ⟹ the forward-backward algorithm.

- **Inference**: Find the optimal parameter $\theta^*$ for a given sequence of observations. ⟹ the Baum–Welch algorithm.

- **Decoding**: Find the most-likely sequence of
hidden states, given a sequence of observations, $\arg\min_{z^{(1:N)}}P(z^{(1:N)}|\mathbf{x}^{(1:N)},\theta)$. ⟹ the Viterbi algorithm.

## Likelihood of HMM

### Forward probability

The likelihood $\ell(\theta)=P(\mathbf{x}^{(1:N)}|\theta)$. Similarly,
$$P(\mathbf{x}^{(1:t)}|\theta)=\sum_{c=1}^K \underbrace{P(\mathbf{x}^{(1:t)}, z^{(t)}=c|\theta)}_{\triangleq\alpha_t(c)} $$

However, 
$$P(\mathbf{x}^{(1:N)}|\theta)\ne \Pi_{i=1}^N \sum_{c=1}^K \left(p(\mathbf{x}^{(i)}, z^{(i)}=c|\theta) \right)$$
Because these hidden state are not sampled independently!

$\alpha_t(c)$, is also called **forward probability**. 

\begin{align}
\alpha_t(c) &= P(\mathbf{x}^{(1:t)}, z^{(t)}=c|\theta) = \sum_{i=1}^K P(\mathbf{x}^{(1:t)}, z^{(t)}=c, z^{(t-1)}=i|\theta) \\
&=\sum_{i=1}^K P(\mathbf{x}^{(t)}| z^{(t)}=c, z^{(t-1)}=i, \mathbf{x}^{(1:t-1)}, \theta)\cdot P(z^{(t)}=c |z^{(t-1)}=i, \mathbf{x}^{(1:t-1)}, \theta)\cdot P(\mathbf{x}^{(1:t-1)}, z^{(t-1)}=i|\theta) \\
&= \sum_{i=1}^K P(\mathbf{x}^{(t)} |\mu_c, \Sigma_c) M_{ic}\alpha_{t-1}(i)
\end{align}

If we use linear algebra, the recurrsion formular has the following compact form, define $\vec\alpha_t = [\alpha_t(1), \dots, \alpha_t(K)]$, $B(\mathbf{x}^{(t)}) =\text{diag}([P(\mathbf{x}^{(t)} |\mu_1, \Sigma_1), \dots, P(\mathbf{x}^{(t)} |\mu_K, \Sigma_K)]) $, then
$$\vec\alpha_t =\vec\alpha_{t-1} M B(\mathbf{x}^{(t)}) $$
For convenience, set $\vec\alpha_0=\vec\pi$. So the forward probability and likelihood are
$$\vec\alpha_N= \vec \pi M B(\mathbf{x}^{(1)})M B(\mathbf{x}^{(2)})\dots M B(\mathbf{x}^{(N)})$$

$$ \ell(\theta)= \vec\alpha_N \mathbb{1}^\top$$
where $\mathbb{1}$ is a row of ones with length $K$. 




### Backward probability 
Define the **backward probability**, $\beta_t(c) = P(\mathbf{x}^{(t+1:N)}|z^{(t)}=c, \theta)$. Note 
\begin{align}
P(\mathbf{x}^{(t+1:N)}|z^{(t)}=c, \theta) &= \sum_{i=1}^K P(\mathbf{x}^{(t+1:N)}|z^{(t+1)}=i, \theta)P(z^{(t+1)}=i|z^{(t)}=c, \theta) \\
&=\sum_{i=1}^K P(\mathbf{x}^{(t+1)}|z^{(t+1)}=i, \theta)P(z^{(t+1)}=i|z^{(t)}=c, \theta)P(\mathbf{x}^{(t+2:N)}|z^{(t+1)}=i, \theta)
\end{align}
Then 
\begin{align}
\beta_t(c)  = \sum_{i=1}^K P(\mathbf{x}^{(t+1)}|\mu_i, \Sigma_i)M_{ci} \beta_{t+1}(i)
\end{align}
Define $\vec\beta_t =[\beta_t(1),\dots, \beta_t(K)]^T$.

For convenience, set $\vec\beta_T=[1, \dots, 1]^T$, the recursion formula has the following compact form.

$$\vec\beta_{t}=MB(\mathbf{x}^{(t+1)})\vec\beta_{t+1}$$

## Task 1: Evaluation
### Forward-Backward algorithm
The first probability is 
\begin{align}
 \boxed{\gamma_{k}(i)} = P(z^{(i)}=k|\mathbf{x}^{(1:N)}, \theta)&=\frac{P(z^{(i)}=k, \mathbf{x}^{(1:N)}|\theta)}{P(\mathbf{x}^{(1:N)}|\theta)}\\
&=\frac{P(z^{(i)}=k, \mathbf{x}^{(1:i)}|\theta)P(\mathbf{x}^{(i+1:N)}|z^{(i)}=k, \theta)}{P(\mathbf{x}^{(1:N)}|\theta)} \\
&= \frac{\alpha_i(k)\beta_i(k)}{P(\mathbf{x}^{(1:N)}|\theta)} \propto \alpha_i(k)\beta_i(k)
\end{align}
This is because in the hidden markov model, 
$$ P(\mathbf{x}^{(i+1:N)}|z^{(i)}=k, \mathbf{x}^{(1:i)})= P(\mathbf{x}^{(i+1:N)}|z^{(i)}=k).$$



---


The second probability is 
\begin{align}
\boxed{\xi_{kc}(i)}&= P(z^{(i)}=k, z^{(i+1)}=c|\mathbf{x}^{(1:N)} ,\theta) \\
&=\frac{P(z^{(i)}=k, z^{(i+1)}=c,\mathbf{x}^{(1:N)}|\theta )}{P(\mathbf{x}^{(1:N)}|\theta )} \\
& = \frac{P(z^{(i)}=k, \mathbf{x}^{(1:i)}|\theta)P(\mathbf{x}^{(i+2:N)}|z^{(i+1)}=c, \theta)P(z^{(i+1)}=c|z^{(i)}=k)P(\mathbf{x}^{(i+1)}|z^{(i+1)}=c)}{P(\mathbf{x}^{(1:N)}|\theta )} \\
&=\frac{\alpha_i(k)\beta_{i+1}(c)M_{kc}P(\mathbf{x}^{(i+1)}|\mu_c, \Sigma_c)}{P(\mathbf{x}^{(1:N)}|\theta)} \propto \alpha_i(k)\beta_{i+1}(c)M_{kc}P(\mathbf{x}^{(i+1)}|\mu_c, \Sigma_c)
\end{align}
This is because in the hidden markov model,  
\begin{align}
P(z^{(i)}, z^{(i+1)},\mathbf{x}^{(1:N)} ) &=P(z^{(i)}, \mathbf{x}^{(1:i)})P(\m{x}^{(i+1:N)}, z^{(i+1)}|z^{(i)}) \\
& =P(z^{(i)}, \mathbf{x}^{(1:i)}) P(z^{(i+1)}|z^{(i)})P(\m{x}^{(i+1:N)}|z^{(i+1)})  \\
&=P(z^{(i)}, \mathbf{x}^{(1:i)}) P(z^{(i+1)}|z^{(i)})P(\m{x}^{(i+1)}|z^{(i+1)})P(\m{x}^{(i+2:N)}|z^{(i+1)}) 
\end{align}

### Log-sum-exp trick
In practice, naively implementing forward algorithm will never work due to arithmetic underflow or overflow. 

Define $g_t(c)=\log \alpha_t(c)$, then 
\begin{align}
 g_t(c) = \log \alpha_t(c) = \log \sum_{i=1}^K \exp\left(l(z^{(t)}=c|z^{(t-1)}=i) + l(\mathbf{x}^{(t)}|z^{(t)}=c)+g_{t-1}(i)\right)
\end{align}
denoting $l=\log P$

To calculate this, one has to use **log-sum-exp** trick:

let’s suppose we would like to compute $\log\sum_{i=1}^m\exp(a_i)$, 

- choose $b=\max_{i}a_i$.

- 
\begin{align}
\log\sum_{i=1}^m\exp(a_i) =\log\sum_{i=1}^m\exp(a_i-b)\exp(b)  = b+\log\sum_{i=1}^m\exp(a_i-b)
\end{align}




---


## Expectation-Maximization(EM) algorithm: the general version
Please refer to [GMM note](https://github.com/yexf308/MAT592/blob/main/Module2/GMM2.ipynb) for special case of EM. 
### The general version
The goal of EM is to find a maximum likelihood estimate (MLE) estimate in models involving hidden/latent/unobserved variables/data.

- Observed data $\mathbf{x}^{(1:N)}$.

- Model $(X, Z)\sim P(\mathbf{x}^{(1:N)} ,z^{(1:N)} |\theta)$. Here $z^{(1:N)}$ represents some collection of unobserved variables. Note EM works
best when $P(\mathbf{x}^{(1:N)} ,z^{(1:N)} |\theta)$ is an exponential family. 

- **Goal:** Find $\theta^*=\arg\max_{\theta} P(\mathbf{x}^{(1:N)}|\theta)$. Note the likelihood $P(\mathbf{x}^{(1:N)}|\theta) = \sum_{z^{(1:N)}}P(\mathbf{x}^{(1:N)} ,z^{(1:N)} |\theta)$. 

- **EM Algorithm:**
 - Initialize $\theta^1$. 

 - For $t=1, 2, \dots$ until some convergence criterion is met,

    - E-step: Compute the  auxiliary function

    \begin{align} \boxed{Q(\color{blue}\theta, \color{red}{\theta^t})=\mathbb{E}_{Z|X, \color{red}{\theta^t}}\left[\log P(X, Z|X=\mathbf{x}^{(1:N)} ,\color{blue}\theta)\right]   = \sum_{z^{(1:N)}} P(z^{(1:N)} |\mathbf{x}^{(1:N)} ,\color{red}{\theta^t})\log P(\mathbf{x}^{(1:N)} ,z^{(1:N)} |\color{blue} \theta)}
    \end{align}

    - M step: Solve for $\color{red}{\theta_{t+1}} = \arg\max_\theta Q(\color{blue}\theta, \color{red}{\theta^t})$

- In practice, we will be able to represent $Q(\color{blue}\theta, \color{red}{\theta^t})$  analytically as a function $\theta$. Moreover, we will
often be able to analytically maximize $Q(\color{blue}\theta, \color{red}{\theta^t})$. 

- It is usually a good idea to introduce some randomization into the initialization $\theta^1$. 





### Proof of correctness

Expectation-maximization works to improve $Q(\color{blue}\theta, \color{red}{\theta^t})$ rather than directly improving the log likelihood $\log P(\mathbf{x}^{(1:N)}|\color{blue}\theta)$. Here it is shown that improvements to the former imply improvements to the latter.

By Baye's rule, for any $z^{(1:N)}$ with non-zero probability  $P(z^{(1:N)}|\mathbf{x}^{(1:N)}, \color{blue}\theta)$, 

$$\log P(\mathbf{x}^{(1:N)}|\color{blue}\theta) = \log P(\mathbf{x}^{(1:N)}, z^{(1:N)}|\color{blue}\theta) - \log P(z^{(1:N)}|\mathbf{x}^{(1:N)}, \color{blue}\theta) $$

By multiplying both sides $P(z^{(1:N)}|\mathbf{x}^{(1:N)}, \color{red}{\theta^t})$ and summing over $z^{(1:N)}$, (which is equvilently, take the expectation over possible values of the hidden variable $Z$ under the current parameter estimate $\color{red}{\theta^t}$)
\begin{align}
\log P(\mathbf{x}^{(1:N)}|\color{blue}\theta)&=\sum_{z^{(1:N)}}P(z^{(1:N)}|\mathbf{x}^{(1:N)}, \color{red}{\theta^t})\log P(\mathbf{x}^{(1:N)}, z^{(1:N)}|\color{blue}\theta) -\sum_{z^{(1:N)}} P(z^{(1:N)}|\mathbf{x}^{(1:N)}, \color{red}{\theta^t})\log P(z^{(1:N)}|\mathbf{x}^{(1:N)}, \color{blue}\theta) \\
& =Q(\color{blue}\theta, \color{red}{\theta^t}) +H(\color{blue}\theta|\color{red}{\theta^t})
\end{align}

when $\theta= \theta^t$, 
$$\log P(\mathbf{x}^{(1:N)}|\color{red}{\theta^t})=Q(\color{red}{\theta^{t}}, \color{red}{\theta^t}) +H(\color{red}{\theta^t}|\color{red}{\theta^t}) $$

 Subtracting this last equation from the previous equation gives
\begin{align}
 \log P(\mathbf{x}^{(1:N)}|\color{blue}\theta) -\log P(\mathbf{x}^{(1:N)}|\color{red}{\theta^t}) &=  Q(\color{blue}\theta, \color{red}{\theta^t}) - Q(\color{red}{\theta^{t}}, \color{red}{\theta^t}) +H(\color{blue}\theta|\color{red}{\theta^t})-H(\color{red}{\theta^t}|\color{red}{\theta^t})   
  \end{align}
  \begin{align}
 \boxed{\log P(\mathbf{x}^{(1:N)}|\color{blue}\theta) -\log P(\mathbf{x}^{(1:N)}|\color{red}{\theta^t}) \ge Q(\color{blue}\theta, \color{red}{\theta^t}) - Q(\color{red}{\theta^{t}}, \color{red}{\theta^t})}
 \end{align}

 It is due to the Gibbs' inequality that $H(\color{blue}\theta|\color{red}{\theta^t})\ge H(\color{red}{\theta^t}|\color{red}{\theta^t})$.

 So choose $\theta$ to maximize $Q(\color{blue}\theta, \color{red}{\theta^t})$ causes $\log P(\mathbf{x}^{(1:N)}|\color{blue}\theta)$ improves at least that much. 

## Task 2: EM for HMM: Baum–Welch algorithm

### E-step
The joint probability $P(\mathbf{x}^{(1:N)} ,z^{(1:N)} |\theta)$ is 
\begin{align}
P(\mathbf{x}^{(1:N)} ,z^{(1:N)} |\theta)&=P(z^{(1)})\cdot\Pi_{i=1}^{N-1}P(z^{(i+1)}|z^{(i)},\theta) \cdot\Pi_{j=1}^N P(\mathbf{x}^{(j)}|z^{(j)},\theta) \\
&=\pi_{z^{(1)}}\cdot\Pi_{i=1}^{N-1} M_{z^{(i)}z^{(i+1)}} \cdot \Pi_{j=1}^N P(\mathbf{x}^{(j)}|\mu_{z^{(j)}},\Sigma_{z^{(j)}})
\end{align}
Then 
\begin{align}
\log P(\mathbf{x}^{(1:N)} ,z^{(1:N)} |\theta) = \log \pi_{z^{(1)}} + \sum_{i=1}^{N-1} \log M_{z^{(i)}z^{(i+1)}} + \sum_{j=1}^N \log P(\mathbf{x}^{(j)}|\mu_{z^{(j)}},\Sigma_{z^{(j)}})
\end{align}

The  auxiliary function $Q(\color{blue}\theta, \color{red}{\theta^t})$ is,

\begin{align}
Q(\color{blue}\theta, \color{red}{\theta^t}) &= \sum_{z^{(1:N)}} P(z^{(1:N)} |\mathbf{x}^{(1:N)} ,\color{red}{\theta^t}) \log P(\mathbf{x}^{(1:N)} ,z^{(1:N)} |\color{blue}\theta)  \\
&= \sum_{z^{(1)}}\boxed{P(z^{(1)}|\mathbf{x}^{(1:N)} ,\color{red}{\theta^t})}\log \pi_{z^{(1)}}+\sum_{i=1}^{N-1}\sum_{z^{(i)}z^{(i+1)}} \boxed{P(z^{(i:i+1)}|\mathbf{x}^{(1:N)} ,\color{red}{\theta^t}) }\log M_{z^{(i)}z^{(i+1)}} + \sum_{j=1}^N \sum_{z^{(j)}}\boxed{P(z^{(j)}|\mathbf{x}^{(1:N)} ,\color{red}{\theta^t})}\log P(\mathbf{x}^{(j)}|\mu_{z^{(j)}},\Sigma_{z^{(j)}}) \\
& =  \sum_{k=1}^K\color{red}{\gamma_{k}^t(1)}\log \color{blue}\pi_{k}+\sum_{i=1}^{N-1}\sum_{k,c=1}^K \color{red}{\xi_{kc}^t(i)}\log\color{blue}{M_{kc}} + \sum_{j=1}^N \sum_{k=1}^K \color{red}{\gamma_{k}^t(j)}\log P(\mathbf{x}^{(j)}|\color{blue} {\mu_{k},\Sigma_{k}})
\end{align}

where $\color{red}{\gamma_{k}^t(i)} = P(z^{(i)}=k|\mathbf{x}^{(1:N)} ,\color{red}{\theta^t})$ and $\color{red}{\xi_{kc}^t(i)}= P(z^{(i)}=k, z^{(i+1)}=c|\mathbf{x}^{(1:N)} ,\color{red}{\theta^t}) $. How to find $\gamma_{k}(i)$ and $\xi_{kc}(i)$?



### M-step 
 For the M-step, we need to find a value of $\theta$ maximizing $Q(\theta, \theta^t)$.  Fortunately, it turns out that we can often do this analytically. 

 - First, to maximize with respect to $\mu_k$ and $\Sigma_k$, i.e., 
 $$ 0= \nabla_{\mu_k}Q(\color{blue}\theta, \color{red}{\theta^t}) = \sum_{j=1}^N \color{red}{\gamma_{k}^t(j)} \nabla_{\mu_k} \log P(\mathbf{x}^{(j)}|\color{blue}{\mu_{k},\Sigma_{k}}) $$

  $$ 0= \nabla_{\Sigma_k}Q(\color{blue}\theta, \color{red}{\theta^t}) = \sum_{j=1}^N \color{red}{\gamma_{k}^t(j)} \nabla_{\Sigma_k} \log P(\mathbf{x}^{(j)}|\color{blue}{\mu_{k},\Sigma_{k}}) $$

  Then we have the analytical solution which is the solution of weighted MLE
  $$\mu_k^{t+1} = \frac{\sum_{j=1}^N \color{red}{\gamma_{k}^t(j)} \mathbf{x}^{(j)}}{\sum_{j=1}^N \color{red}{\gamma_{k}^t(j)}} $$

$$\Sigma_k^{t+1} =\frac{\sum_{j=1}^N \color{red}{\gamma_{k}^t(j)} \left[\mathbf{x}^{(i)}-\mu_k^{t+1}\right]\left[\mathbf{x}^{(i)}-\mu_k^{t+1}\right]^T }{\sum_{j=1}^N \color{red}{\gamma_{k}^t(j)}} $$

- Second, to maximize with respect to $\vec\pi$. The constraint is $\sum_{k}\pi_k = 1$. We can do this analytically using the method of Lagrange multipliers, the result is 
 
 $$ \pi_k^{t+1} =\frac{\gamma_k^t(1)}{\sum_{c=1}^K \gamma_c^t(1)} $$


- Third, to maximize with respect to the Markov transition matrix $M$. The contraint is $\sum_{c=1}^K M_{kc} =1$. We can do this analytically using
Lagrange multipliers as well. 

$$M_{kc}^{t+1}= \frac{\sum_{j=2}^N \xi_{kc}^t(j)}{\sum_{j=2}^N \sum_{c=1}^K \xi_{kc}^t(j)}. $$









If we look at the MLE, $\gamma_k(i)$ shows up in both denorminator and numerator. So what matters is the relative ratio $\gamma_k(i)$ for different $i$. 

So we can renormalized $\vec\alpha_i$ and $\vec\beta_i$ at each $i$. 










---

## Task 3: Decoding - Viterbi algorithm
Some backgound information. 
- If $c\ge 0, f(x)\ge 0$, then $\max_x cf(x)=c\max_xf(x)$. 

- $\max_{xy} f(x,y)=\max_x\max_y f(x,y)$. 

The goal of the Viterbi algorithm is to find a
\begin{align}
z_{1:N}^* = \arg\max_{z_{1:N}} P(z_{1:N}|\m{x}_{1:N})
\end{align}
Note $P(\m{x}_{1:N})$ is constant with respect to $z_{1:N}$, this  is equivalent to 

\begin{align}
z_{1:N}^* = \arg\max_{z_{1:N}} P(z_{1:N},\m{x}_{1:N})
\end{align}

Define $\mu_1(z_1)=p(z_1)p(\m{x}_1|z_1)$. Writing
out the factorization implied by the graphical model for an HMM,
\begin{align}
P(z_{1:N},\m{x}_{1:N}) = \underbrace{P(z_1)P(\m{x}_1|z_1)}_{=\mu_1(z_1)} P(z_2|z_1) P(\m{x}_2|z_2) \Pi_{t=3}^N P(z_{t}|z_{t-1}) P(\m{x}_t|z_t)
\end{align}
We have
\begin{align}
\max_{z_{1:N}} P(z_{1:N},\m{x}_{1:N}) &= \max_{z_{2:N}}\left(\underbrace{\max_{z_1} \mu_1(z_1)  P(z_2|z_1) P(\m{x}_2|z_2) }_{=\mu_2(z_2)}\right)\Pi_{t=3}^N P(z_{t}|z_{t-1}) P(\m{x}_t|z_t) \\
&=\max_{z_{3:N}}\left(\underbrace{\max_{z_2} \mu_2(z_2)  P(z_3|z_2) P(\m{x}_3|z_3) }_{=\mu_3(z_3)}\right)\Pi_{t=4}^N P(z_{t}|z_{t-1}) P(\m{x}_t|z_t) \\
&=\max_{z_{j:N}}\left(\underbrace{\max_{z_{j-1}} \mu_{j-1}(z_{j-1})  P(z_j|z_{j-1}) P(\m{x}_{j}|z_j) }_{=\mu_j(z_j)}\right)\Pi_{t=j+1}^N P(z_{t}|z_{t-1}) P(\m{x}_t|z_t) = \max_{z_N}\mu_N(z_N)
\end{align}




### Algorithm for  $\arg\max_{z_{1:N}} P(z_{1:N},\m{x}_{1:N})$ 
we can compute $\arg\max_{z_{1:N}} P(z_{1:N},\m{x}_{1:N})$ via the following algorithm:

- For each $z_1= 1,\dots, K$, compute $\mu_1(z_1)= P(z_1)P(\m{x}_1|z_1) \qquad O(K)$

  and record the optimal $z_1^*=\arg\max_{z_1}P(z_1)P(\m{x}_1|z_1)$. 

- For each $j=2, \dots, N$, for each $z_j=1,\dots, K$, compute
\begin{align}
\mu_j(z_j) =\max_{z_{j-1}} \mu_{j-1}(z_{j-1})  P(z_j|z_{j-1}) P(\m{x}_{j}|z_j)  \qquad O(NK^2)
\end{align}
and record the optimal $z_{j-1}^*$ as 
\begin{align}
z_{j-1}^* =\arg\max_{z_{j-1}} \mu_{j-1}(z_{j-1})  P(z_j|z_{j-1}) P(\m{x}_{j}|z_j)
\end{align}

- Compute $ \max_{z_N}\mu_N(z_N)$ and $z_N^* = \arg\max_{z_N} \mu_N(z_N)$. $\qquad O(K)$


---



We sequentially obtained the optimal sequence $z_{1:N}^*$, so need to verify it actually attains the maximum. Note for each $j=N, N-1,\dots, 2$,
$$ \mu_j(z_j^*) =\max_{z_{j-1}} \mu_{j-1}(z_{j-1})  P(z_j^*|z_{j-1}) P(\m{x}_{j}|z_j^*) $$
We can plug in the expression repeatly on $\mu_N(z_N^*)$, 
\begin{align}
\mu_N(z_N^*) = P(z^*_{1:N},\m{x}_{1:N} )=\max_{z^{1:N}}P(z_{1:N},\m{x}_{1:N} )
\end{align}


---
###  Fixing arithmetic underflow/overflow
Note $\mu_j(z_j^*)$ is usually very small number since it multiply the probability more than hundreds times so the value may
have "arithmetic underflow", i.e., the value is too small and it is considered as zero. 

The key is to log probability!

Define $\ell= \log P$, i.e., $\ell(z_1)=\log P(z_1), \ell(z_t|z_{t-1})=\log P(z_t|z_{t-1})$ and $\ell(\m{x}_t|z_t)=\log P(\m{x}_t|z_t)$.

The algorithm described above works if we use $f_i(z_j)$ in place of $\log\mu_j(z_j)$,
\begin{align}
&f_1(z_1) = \ell(z_1) +\ell(\m{x}_1|z_1) \\
&f_j(z_j) = \max_{z_{j-1}}\left( f_{j-1}(z_{j-1}) +\ell(z_j|z_{j-1}) +\ell(\m{x}_j|z_j) \right)
\end{align}

The reason why it works is because log is order-preserving.. 
Consequently, choosing $z_j^*$ in this way is equivalent to the earlier way. 
